# Loading the `Moral-Stories` dataset
***
The dataset and code can be found <a href="https://github.com/demelin/moral_stories">here</a>.\
The authors provide 12k unique norms and, for some reason, additional 700k variations of the same norms, just with NaN fields every now and then. Zero additional information, but maybe I am overlooking something here?
* Might be for different tasks? But then they only provide a single label which is always 1 for any NaN rows...

# Sample task: Action classification
***
For starters, let's reproduce a task from the paper:
* Given an action, predict whether it is moral or immoral.
* For simplicity, we do not use the splits introduced in the paper, but rather random splitting

We start by loading the data as a `pandas.DataFrame`:

In [3]:
from ailignment.datasets.moral_stories import get_moral_stories, make_action_classification_dataframe
dataframe = get_moral_stories()
action_dataframe = make_action_classification_dataframe(dataframe)

## Task 1: Action only
***
We'll only give single sentences to the model for now. Let's start by feeding the actions.

In [3]:
def tokenize_and_split(dataset, tokenizer, sentence_col="text"):
    '''
    Takes a `datasets.Dataset` with train and test splits
    and applies the given tokenizer.
    Returns tokenized train and test split datasets
    '''
    def tokenize_function(examples):
        return tokenizer(examples[sentence_col], padding="max_length", truncation=True)

    tokenized_dataset = dataset.map(tokenize_function, batched=True)
    train_dataset = tokenized_dataset["train"]
    eval_dataset = tokenized_dataset["test"]
    return train_dataset, eval_dataset

from datasets import load_metric
import numpy as np
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [4]:
import datasets
test_split = 0.2
batch_size = 8

action_dataframe["task_input"] = action_dataframe.drop(["ID", "labels"], axis=1).agg(". ".join, axis=1)
dataset = datasets.Dataset.from_pandas(action_dataframe)
dataset = dataset.train_test_split(test_size=test_split)

In [5]:
from transformers import (
    AutoModelForSequenceClassification, DistilBertTokenizerFast,
     Trainer, TrainingArguments, AutoModelWithLMHead, AutoTokenizer,
)
import torch

model = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForSequenceClassification.from_pretrained(model)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'classifier

In [6]:
train_data, test_data = tokenize_and_split(dataset, tokenizer, "task_input")

  0%|          | 0/20 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [ ]:
small_train_dataset = train_data.shuffle(seed=42).select(range(1000))
small_eval_dataset = test_data.shuffle(seed=42).select(range(1000))

In [7]:
training_args = TrainingArguments(
    output_dir="results/",
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=12,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=50,                # how often to log
    save_steps=1000,
    save_total_limit=1,
    evaluation_strategy="epoch",     # when to run evaluation
)

In [8]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_data,   # training dataset
    eval_dataset=test_data,     # evaluation dataset
    compute_metrics=compute_metrics,     # code to run accuracy metric
)
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: action, intention, consequence, situation, norm, ID, task_input.
***** Running training *****
  Num examples = 19198
  Num Epochs = 5
  Instantaneous batch size per device = 12
  Total train batch size (w. parallel, distributed & accumulation) = 12
  Gradient Accumulation steps = 1
  Total optimization steps = 8000


Epoch,Training Loss,Validation Loss,Accuracy
1,0.181500,0.126275,0.956667
2,0.127200,0.116812,0.966042
3,0.038400,0.179140,0.962500
4,0.039400,0.210755,0.966875
5,0.013900,0.227468,0.968333


Saving model checkpoint to results/checkpoint-1000
Configuration saved in results/checkpoint-1000\config.json
Model weights saved in results/checkpoint-1000\pytorch_model.bin
Deleting older checkpoint [results\checkpoint-7000] due to args.save_total_limit
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: action, intention, consequence, situation, norm, ID, task_input.
***** Running Evaluation *****
  Num examples = 4800
  Batch size = 8
Saving model checkpoint to results/checkpoint-2000
Configuration saved in results/checkpoint-2000\config.json
Model weights saved in results/checkpoint-2000\pytorch_model.bin
Deleting older checkpoint [results\checkpoint-1000] due to args.save_total_limit
Saving model checkpoint to results/checkpoint-3000
Configuration saved in results/checkpoint-3000\config.json
Model weights saved in results/checkpoint-3000\pytorch_model.bin
Deleting older checkpoint

TrainOutput(global_step=8000, training_loss=0.07894983084814157, metrics={'train_runtime': 4269.1794, 'train_samples_per_second': 22.484, 'train_steps_per_second': 1.874, 'total_flos': 1.97437790512128e+16, 'train_loss': 0.07894983084814157, 'epoch': 5.0})

# WIP: Get score output from LM
***
Question: Is there a better way to sample from generated LM outputs?

In [ ]:
from transformers import (
    AutoModelForSequenceClassification, DistilBertTokenizerFast,
     Trainer, TrainingArguments, AutoModelWithLMHead, AutoTokenizer,
)
import torch

model = "distilbert-base-uncased"
model = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelWithLMHead.from_pretrained(model)

prompt = "Today the weather is really nice and I am planning on "
inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

prompt_length = len(tokenizer.decode(inputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))
outputs = model.generate(inputs, max_length=250, do_sample=False, top_p=0.95, top_k=60,
                        return_dict_in_generate=True, output_attentions=False,
                        output_hidden_states=True, output_scores=True)
#generated = prompt + tokenizer.decode(outputs[0])[prompt_length:]

p = torch.softmax(outputs.scores[0], dim=1)

print(p.max())